# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from contextlib import redirect_stdout
import numpy as np
import os
import pandas as pd
import plotly.express as px

from cost import *
from plotting import *
from prices import *
from inflation import *
from regression import *
from utils import *

In [3]:
estimation_method = 'hardware-capex-energy'  # hardware-capex-energy, hardware-acquisition, cloud
compute_threshold_method = 'top_n'  # top_n, window_percentile
compute_threshold = 10  # e.g. 10 to select top 10; 75 to select top 25%
variant = 'update-2025-03-13'  # whatever else distinguishes this run, e.g. 'excluding-AlphaGo'
exclude_models_containing = []  # ['GNMT', 'AlphaZero', 'AlphaGo Master', 'AlphaGo Zero']

estimation_method_lookup = {
    'hardware-capex-energy': estimate_hardware_capex_energy,
    'hardware-acquisition': estimate_hardware_acquisition_cost,
    'cloud': estimate_cloud_costs,
}
cost_estimation_function = estimation_method_lookup[estimation_method]

results_dir = f'results/{estimation_method}-{compute_threshold_method}={compute_threshold}-{variant}/'
os.makedirs(results_dir, exist_ok=True)

# Load data

In [4]:
frontier_pcd_df, hardware_df, price_df = load_data_for_cost_estimation(
    compute_threshold_method=compute_threshold_method, compute_threshold=compute_threshold,
)

In [5]:
len(frontier_pcd_df), len(hardware_df), len(price_df)

(89, 5703, 576)

# Cost estimation

In [6]:
with open(f'{results_dir}/cost_estimation.out', 'w') as f:
    with redirect_stdout(f):
        cost_df = cost_estimation_function(frontier_pcd_df, hardware_df, price_df)

In [7]:
if estimation_method == 'hardware-capex-energy':
    frontier_pcd_df_copy = frontier_pcd_df.copy()
    with open(f'{results_dir}/component_cost_estimation.out', 'w') as f:
        with redirect_stdout(f):
            component_cost_df = cost_estimation_function(frontier_pcd_df_copy, hardware_df, price_df, separate_components=True)

In [8]:
cost_df

,Model,Domain,Task,Authors,Notability criteria,Notability criteria notes,Model accessibility,Link,Citations,Reference,...,Post-training compute (FLOP),Post-training compute notes,Hardware maker,benchmarks/models,Maybe over 1e25 FLOP,Updated dataset size,WT103 ppl,WT2 ppl,PTB ppl,Cost
109,Doubao-pro,Language,"Language modeling/generation,Question answerin...",NaN,Training cost,NaN,API access,https://www.volcengine.com/docs/6360/1264663,NaN,Doubao General Model Pro (Doubao-pro),...,NaN,NaN,NaN,doubao-pro-32k,NaN,NaN,NaN,NaN,NaN,NaN
312,GLM-4-Plus,Language,Language modeling,Zhipu AI,Training cost,NaN,API access,https://bigmodel.cn/dev/howuse/glm-4,NaN,GLM-4-Plus,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
335,Grok-2,"Language,Vision,Multimodal","Chat,Language modeling/generation,Question ans...",NaN,Training cost,NaN,Hosted access (no API),https://x.ai/blog/grok-2,NaN,Grok-2 Beta Release,...,NaN,NaN,NVIDIA,"grok-2-1212,grok-2-vision-1212,grok-2-0813",NaN,NaN,NaN,NaN,NaN,3.069502e+07
367,Mistral Large 2,Language,"Language modeling/generation,Translation,Code ...","Albert Jiang, Alexandre Sablayrolles, Alexis T...",Training cost,likely high training cost since previous Mistr...,Open weights (non-commercial),https://mistral.ai/news/mistral-large-2407/,NaN,"Top-tier reasoning for high-complexity tasks, ...",...,NaN,NaN,NaN,"Mistral-Large-Instruct-2407,Mistral-Large-Inst...",NaN,NaN,NaN,NaN,NaN,NaN
370,Llama 3.1-405B,Language,Language modeling/generation,"Abhimanyu Dubey, Abhinav Jauhri, Abhinav Pande...","SOTA improvement,Training cost","High training compute, exceeds 4o and Claude 3...",Open weights (restricted use),https://ai.meta.com/research/publications/the-...,NaN,The Llama 3 Herd of Models,...,9.400000e+22,Section 4 gives detail about the post-training...,NVIDIA,Llama-3.1-405B-Instruct,NaN,NaN,NaN,NaN,NaN,5.138954e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2054,AlphaGo Lee,Games,Go,"David Silver, Aja Huang, Chris J. Maddison, Ar...",Highly cited,NaN,Unreleased,https://www.nature.com/articles/nature16961,16057.0,Mastering the game of Go with deep neural netw...,...,NaN,NaN,NaN,NaN,NaN,0 checked out of 1,NaN,NaN,NaN,NaN
2058,ResNet-152 (ImageNet),Vision,Image classification,"Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun",Highly cited,NaN,NaN,https://arxiv.org/abs/1512.03385,175697.0,Deep Residual Learning for Image Recognition,...,NaN,NaN,NaN,NaN,NaN,0 checked out of 1,NaN,NaN,NaN,NaN
2060,ResNet-101 (ImageNet),Vision,Image classification,"Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun",Highly cited,NaN,Open weights (unrestricted),https://arxiv.org/abs/1512.03385,175697.0,Deep Residual Learning for Image Recognition,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2061,DeepSpeech2 (English),Speech,Speech recognition,"Dario Amodei, Rishita Anubhai, Eric Battenberg...",Highly cited,NaN,NaN,https://arxiv.org/abs/1512.02595,2853.0,Deep Speech 2: End-to-End Speech Recognition i...,...,NaN,NaN,NVIDIA,NaN,NaN,0 checked out of 1,NaN,NaN,NaN,1.854566e+02


In [9]:
cost_df['Cost'].notna().sum()

55

In [10]:
cost_df.dropna(subset=['Cost'])['Training time (hours)'].notna().sum()

36

In [11]:
cost_df.dropna(subset=['Cost'])['Hardware utilization'].notna().sum()

20

Exclusion

In [12]:
cost_df[['Model', 'Publication date']].tail(15)

,Model,Publication date
1920,Libratus,2017-08-19
1929,OpenAI TI7 DOTA 1v1,2017-08-11
1935,ConvS2S (ensemble of 8 models),2017-07-25
1941,JFT,2017-07-10
1965,MoE-Multi,2017-01-23
1985,PolyNet,2016-11-17
1991,NASv3 (CIFAR-10),2016-11-05
2000,Xception,2016-10-07
2001,GNMT,2016-09-26
2013,ResNet-200,2016-09-17


In [13]:
for kw in exclude_models_containing:
    cost_df = cost_df[cost_df['Model'].str.contains(kw) == False]
cost_df[['Model', 'Publication date']].tail(15)

,Model,Publication date
1920,Libratus,2017-08-19
1929,OpenAI TI7 DOTA 1v1,2017-08-11
1935,ConvS2S (ensemble of 8 models),2017-07-25
1941,JFT,2017-07-10
1965,MoE-Multi,2017-01-23
1985,PolyNet,2016-11-17
1991,NASv3 (CIFAR-10),2016-11-05
2000,Xception,2016-10-07
2001,GNMT,2016-09-26
2013,ResNet-200,2016-09-17


Use the below to check data availability for specific systems

In [14]:
# system = 'WizardLM-7B'
# row = cost_df.loc[cost_df['Model'] == system]
# print('Cost:', row['Cost'].values[0])
# print('Training hardware:', row['Training hardware'].values[0])
# print('Training time (hours):', row['Training time (hours)'].values[0])
# print('Hardware quantity:', row['Hardware quantity'].values[0])
# print('Hardware utilization:', row['Hardware utilization'].values[0])

# Apply inflation adjustment

In [15]:
cost_df['Cost'].dropna()

335     3.069502e+07
370     5.138954e+07
417     2.063627e+07
592     1.182473e+07
618     1.416386e+07
624     2.634254e+06
633     7.839204e+06
732     2.840641e+07
757     1.292526e+07
843     7.589215e+06
865     1.027732e+07
1010    4.603240e+06
1075    4.015593e+07
1152    4.534422e+06
1249    3.266061e+05
1288    7.202773e+05
1299    2.821356e+06
1322    2.224292e+05
1355    5.112708e+05
1359    5.975319e+05
1403    3.590762e+06
1411    5.674315e+05
1427    7.803260e+05
1456    8.940160e+04
1465    8.287237e+04
1505    5.211218e+04
1522    1.345841e+05
1525    1.209177e+05
1590    2.456145e+05
1599    1.000088e+05
1606    2.088539e+06
1647    1.968088e+05
1654    3.920096e+06
1655    3.013336e+05
1669    4.119750e+04
1673    7.570550e+04
1675    1.180148e+05
1678    1.573545e+04
1679    7.087411e+04
1693    1.670119e+05
1694    1.061334e+05
1720    8.218969e+04
1741    1.218152e+04
1746    8.843100e+03
1780    4.161844e+03
1812    4.799969e+03
1854    1.288594e+05
1905    3.277

In [16]:
cost_df = adjust_column_for_inflation(cost_df, 'Cost', 'data/PCU518210518210.csv', '2024-12-01')

In [17]:
cost_df['Cost (inflation-adjusted)'].dropna()

335     3.058916e+07
370     5.116917e+07
417     2.058539e+07
592     1.183475e+07
618     1.414371e+07
624     2.630507e+06
633     7.828054e+06
732     2.848278e+07
757     1.299024e+07
843     7.656705e+06
865     1.036871e+07
1010    4.663057e+06
1075    4.069571e+07
1152    4.637986e+06
1249    3.336625e+05
1288    7.336346e+05
1299    2.875763e+06
1322    2.305682e+05
1355    5.290141e+05
1359    6.182688e+05
1403    3.714250e+06
1411    5.879853e+05
1427    8.075117e+05
1456    9.270193e+04
1465    8.593166e+04
1505    5.398904e+04
1522    1.400390e+05
1525    1.258187e+05
1590    2.569136e+05
1599    1.046095e+05
1606    2.198105e+06
1647    2.073162e+05
1654    4.177318e+06
1655    3.211059e+05
1669    4.401863e+04
1673    8.088967e+04
1675    1.260962e+05
1678    1.681299e+04
1679    7.572743e+04
1693    1.786084e+05
1694    1.135028e+05
1720    8.869130e+04
1741    1.316895e+04
1746    9.577270e+03
1780    4.503281e+03
1812    5.184357e+03
1854    1.393047e+05
1905    3.553

In [18]:
# Equal number of non-null values
assert cost_df['Cost (inflation-adjusted)'].notna().sum() == cost_df['Cost'].notna().sum()

# Regression

In [19]:
cost_df['Publication date (float)'] = datetime_to_float_year(pd.to_datetime(cost_df['Publication date']))

In [20]:
reg_results = fit_ols_regression(cost_df, ['Publication date (float)'], 'Cost (inflation-adjusted)', logy=True)
reg_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.707
Model:                            OLS   Adj. R-squared:                  0.701
Method:                 Least Squares   F-statistic:                     127.7
Date:                Thu, 13 Mar 2025   Prob (F-statistic):           9.79e-16
Time:                        17:02:37   Log-Likelihood:                -55.849
No. Observations:                  55   AIC:                             115.7
Df Residuals:                      53   BIC:                             119.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -896.4478     79.805    -11.233      0.000   -1056.516    -736.379
x1             0.4463      0.039     11.302      0.000       0.367       0.526
==============================================================================
Omnibus:                        3.655   Durbin-Watson:                   1.355
Prob(Omnibus):                  0.161   Jarque-Bera (JB):                3.097
Skew:                           0.581   Prob(JB):                        0.213
Kurtosis:                       3.053   Cond. No.                     1.76e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.76e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [21]:
with open(f'{results_dir}/regression_results.out', 'w') as f:
    with redirect_stdout(f):
        print_growth_rates(reg_results, round_digits=None)
print_growth_rates(reg_results, ci=90, round_digits=5)

N=55.0
R^2=0.71
0.44633 OOMs/year (90% CI: 0.38022, 0.51244)
2.79466x/year (90% CI: 2.40004x, 3.25416x)
doubling time of 8.09349 months (90% CI: 7.04935, 9.50074)


In [22]:
pred_start_year = 2015
pred_end_year = 2025
pred_start_date = f'{pred_start_year}-01-01'
pred_end_date = f'{pred_end_year}-01-01'

pred_years = pd.DataFrame({'Publication date (float)': np.linspace(pred_start_year, pred_end_year, 100)})
pred_years

,Publication date (float)
0,2015.00000
1,2015.10101
2,2015.20202
3,2015.30303
4,2015.40404
...,...
95,2024.59596
96,2024.69697
97,2024.79798
98,2024.89899


In [23]:

predicted_cost_df = get_predictions(reg_results, pred_years, ['Publication date (float)'])
predicted_cost_df['Publication date'] = predicted_cost_df['Publication date (float)'].apply(float_year_to_datetime)
predicted_cost_df

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper,Publication date (float),Publication date
0,2.904894,0.250717,2.485165,3.324623,1.690855,4.118933,2015.00000,2015-01-01
1,2.949978,0.247009,2.536456,3.363500,1.738071,4.161884,2015.10101,2015-02-06
2,2.995061,0.243310,2.587732,3.402391,1.785253,4.204869,2015.20202,2015-03-15
3,3.040145,0.239621,2.638992,3.441298,1.832403,4.247887,2015.30303,2015-04-21
4,3.085229,0.235941,2.690237,3.480221,1.879519,4.290939,2015.40404,2015-05-28
...,...,...,...,...,...,...,...,...
95,7.187848,0.172113,6.899711,7.475985,6.012799,8.362897,2024.59596,2024-08-06
96,7.232932,0.175500,6.939123,7.526740,6.056479,8.409384,2024.69697,2024-09-12
97,7.278015,0.178913,6.978494,7.577536,6.100123,8.455908,2024.79798,2024-10-19
98,7.323099,0.182349,7.017826,7.628372,6.143731,8.502467,2024.89899,2024-11-25


In [24]:
predicted_cost_df.to_csv(results_dir + 'predicted_cost_dataset.csv', index=False)

# Export data

In [25]:
keep_cols = [
    'Model',
    'Domain',
    'Task',
    'Model accessibility',
    'Reference',
    'Publication date',
    'Organization',
    'Parameters',
    'Training compute (FLOP)',
    'Training dataset size (datapoints)',
    'Epochs',
    'Training time (hours)',
    'Training hardware',
    'Country (from Organization)',
    'Base model',
    'Finetune compute (FLOP)',
    'Hardware quantity',
    'Hardware utilization',
    'Training cloud compute vendor',
    'Training data center',
    # 'Training time (chip hours)',
    'Cost',
    'Cost (inflation-adjusted)',
]
cost_df[keep_cols]

,Model,Domain,Task,Model accessibility,Reference,Publication date,Organization,Parameters,Training compute (FLOP),Training dataset size (datapoints),...,Training hardware,Country (from Organization),Base model,Finetune compute (FLOP),Hardware quantity,Hardware utilization,Training cloud compute vendor,Training data center,Cost,Cost (inflation-adjusted)
109,Doubao-pro,Language,"Language modeling/generation,Question answerin...",API access,Doubao General Model Pro (Doubao-pro),2024-10-28,ByteDance,5.000000e+11,2.505000e+25,8.350000e+12,...,NaN,China,NaN,NaN,NaN,NaN,NaN,"There is no paper to reference, also no inform...",NaN,NaN
312,GLM-4-Plus,Language,Language modeling,API access,GLM-4-Plus,2024-08-29,Zhipu AI,NaN,3.600000e+25,NaN,...,NaN,China,NaN,NaN,NaN,NaN,NaN,Check references for hardware details.,NaN,NaN
335,Grok-2,"Language,Vision,Multimodal","Chat,Language modeling/generation,Question ans...",Hosted access (no API),Grok-2 Beta Release,2024-08-13,xAI,NaN,2.960000e+25,NaN,...,NVIDIA H100 SXM5 80GB,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,3.069502e+07,3.058916e+07
367,Mistral Large 2,Language,"Language modeling/generation,Translation,Code ...",Open weights (non-commercial),"Top-tier reasoning for high-complexity tasks, ...",2024-07-24,Mistral AI,1.230000e+11,2.130000e+25,NaN,...,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370,Llama 3.1-405B,Language,Language modeling/generation,Open weights (restricted use),The Llama 3 Herd of Models,2024-07-23,Meta AI,4.050000e+11,3.800000e+25,1.560000e+13,...,NVIDIA H100 SXM5 80GB,United States of America,NaN,NaN,16384.0,0.4042,NaN,NaN,5.138954e+07,5.116917e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2054,AlphaGo Lee,Games,Go,Unreleased,Mastering the game of Go with deep neural netw...,2016-01-27,DeepMind,NaN,1.900000e+21,2.940000e+07,...,NaN,United Kingdom of Great Britain and Northern I...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2058,ResNet-152 (ImageNet),Vision,Image classification,NaN,Deep Residual Learning for Image Recognition,2015-12-10,Microsoft,6.020000e+07,1.041408e+19,1.280000e+06,...,NaN,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2060,ResNet-101 (ImageNet),Vision,Image classification,Open weights (unrestricted),Deep Residual Learning for Image Recognition,2015-12-10,Microsoft,4.450000e+07,7.004000e+18,1.280000e+06,...,NaN,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2061,DeepSpeech2 (English),Speech,Speech recognition,NaN,Deep Speech 2: End-to-End Speech Recognition i...,2015-12-08,Baidu Research - Silicon Valley AI Lab,3.800000e+07,2.600000e+19,1.633392e+08,...,NVIDIA GeForce GTX TITAN X,United States of America,NaN,NaN,16.0,0.4484,NaN,NaN,1.854566e+02,2.068604e+02


In [26]:
cost_df[keep_cols].to_csv(results_dir + 'cost_dataset.csv', index=False)

# Plots

In [27]:
fig = px.scatter(
    cost_df,
    x='Publication date',
    y='Cost (inflation-adjusted)',
    text='Model',
    log_y=True,
)
fig.update_traces(textposition='top center')

# no legend
fig.update_layout(showlegend=False)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2024 USD, log scale)')

# title
fig.update_layout(title_text=get_cost_plot_title(estimation_method, compute_threshold_method, compute_threshold))

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    title_font=dict(
        size=16,
    )
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_scatter')

fig.show()

In [51]:
label_systems = ['GNMT', 'AlphaGo Zero', 'DALL-E', 'GPT-3 175B (davinci)', 'GPT-4', 'Llama 3.1-405B', 'Grok-2']

tpu_mask = cost_df['Training hardware'].str.contains('TPU', na=False)
tpu_cost_df = cost_df.loc[tpu_mask]
gpu_cost_df = cost_df.loc[~tpu_mask]

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=gpu_cost_df['Publication date'],
    y=gpu_cost_df['Cost (inflation-adjusted)'],
    text=gpu_cost_df['Model'],
    mode='markers',
    showlegend=False,
))
fig.update_yaxes(type='log')
fig.add_trace(go.Scatter(
    x=tpu_cost_df['Publication date'],
    y=tpu_cost_df['Cost (inflation-adjusted)'],
    text=tpu_cost_df['Model'],
    mode='markers',
    marker_symbol='circle-open' if estimation_method != 'cloud' else 'circle',
    name='Using estimated cost of TPU' if estimation_method != 'cloud' else '',
    showlegend=estimation_method != 'cloud',
))
fig.add_trace(go.Scatter(
    x=gpu_cost_df.loc[gpu_cost_df['Model'].isin(label_systems)]['Publication date'],
    y=gpu_cost_df.loc[gpu_cost_df['Model'].isin(label_systems)]['Cost (inflation-adjusted)'],
    text=gpu_cost_df.loc[gpu_cost_df['Model'].isin(label_systems)]['Model'],
    mode='text',
    showlegend=False,
))
fig.add_trace(go.Scatter(
    x=tpu_cost_df.loc[tpu_cost_df['Model'].isin(label_systems)]['Publication date'],
    y=tpu_cost_df.loc[tpu_cost_df['Model'].isin(label_systems)]['Cost (inflation-adjusted)'],
    text=tpu_cost_df.loc[tpu_cost_df['Model'].isin(label_systems)]['Model'],
    mode='text',
    showlegend=False,
))

# Marker color
fig.update_traces(
    marker=dict(
        color='rgb(0,100,200)',
    ),
    selector=dict(mode='markers'),
)

# Shade in CI
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_lower'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
)
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean_ci_upper'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(0,100,200,0.2)',
    line=dict(width=0),
    name='90% CI of mean',
)
fig.add_scatter(
    x=predicted_cost_df['Publication date'],
    y=10**predicted_cost_df['mean'],
    mode='lines',
    line=dict(color='rgb(0,100,200)'),
    name=f'Regression mean (growth rate: {10**reg_results.params[1]:.1f}x per year)',
)

fig.update_traces(textposition='top center')

# axis limits
# fig.update_xaxes(range=[pred_start_date, pred_end_date])
fig.update_xaxes(range=['2015-01-01', '2025-06-01'])  # manual
if estimation_method == 'hardware-acquisition':
    fig.update_yaxes(range=[4, 10])
else:
    fig.update_yaxes(range=[1, 9])

# legend on bottom-right of the axes
fig.update_layout(
    legend=dict(
        x=0.45,
        y=0.05,
    )
)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Cost (2024 USD, log scale)')

# title
fig.update_layout(title_text=get_cost_plot_title(estimation_method, compute_threshold_method, compute_threshold))

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    title_font=dict(
        size=16,
    ),
    title_x=0.5,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=60, b=10))

save_plot(fig, results_dir, 'cost_regression')

fig.show()

# Cost components

In [29]:
cost_component_names = [
    'AI accelerator chip cost',
    'Other server components cost',
    'Cluster-level interconnect cost',
    'Energy cost',
]

In [30]:
for key in cost_component_names:
    component_cost_df[f"{key} (%)"] = component_cost_df[key] / component_cost_df['Cost'] * 100
component_cost_df['AI accelerator chip cost (%)']

109           NaN
312           NaN
335     45.955203
367           NaN
370     46.530646
          ...    
2054          NaN
2058          NaN
2060          NaN
2061    34.484219
2069          NaN
Name: AI accelerator chip cost (%), Length: 89, dtype: float64

In [31]:
cost_component_pc_names = [name + ' (%)' for name in cost_component_names]
filtered_component_cost_df = component_cost_df.dropna(subset=cost_component_pc_names).sort_values(by='Publication date')

In [32]:
# Stacked bar chart of cost components, using component_cost_df
fig = px.bar(
    filtered_component_cost_df,
    x='Model',
    y=cost_component_pc_names,
    barmode='stack',
)

# axis labels
fig.update_xaxes(title_text='ML model')
fig.update_yaxes(title_text='% of amortized hardware CapEx + energy')
fig.update_layout(
    legend=dict(
        title_text='Cost component',
        x=0.60,
        y=0.05,
    )
)
# limits 0 to 100
fig.update_yaxes(range=[0, 100])

fig.update_yaxes(tickvals=list(range(0, 101, 10)))

# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'cost_component_percentage')

fig.show()

In [33]:
filtered_component_cost_df.head()

,Model,Domain,Task,Authors,Notability criteria,Notability criteria notes,Model accessibility,Link,Citations,Reference,...,PTB ppl,Cost,AI accelerator chip cost,Other server components cost,Cluster-level interconnect cost,Energy cost,AI accelerator chip cost (%),Other server components cost (%),Cluster-level interconnect cost (%),Energy cost (%)
2061,DeepSpeech2 (English),Speech,Speech recognition,"Dario Amodei, Rishita Anubhai, Eric Battenberg...",Highly cited,NaN,NaN,https://arxiv.org/abs/1512.02595,2853.0,Deep Speech 2: End-to-End Speech Recognition i...,...,NaN,185.456639,63.953273,40.930095,24.602271,55.971000,34.484219,22.069900,13.265781,30.180101
2001,GNMT,Language,Translation,"Yonghui Wu, Mike Schuster, Zhifeng Chen, Quoc ...",Highly cited,NaN,Hosted access (no API),https://arxiv.org/abs/1609.08144,6483.0,Google's Neural Machine Translation System: Br...,...,NaN,177459.232941,77894.044829,49852.188691,29965.165887,19747.833534,43.894050,28.092192,16.885662,11.128096
2000,Xception,Vision,Image classification,François Chollet,Highly cited,NaN,NaN,https://arxiv.org/abs/1610.02357,13038.0,Xception: Deep Learning with Depthwise Separab...,...,NaN,11554.506253,5064.230483,3241.107509,1948.165702,1301.002560,43.829051,28.050593,16.860657,11.259698
1985,PolyNet,Vision,Image classification,"X Zhang, Z Li, C Change Loy",SOTA improvement,"""The Very Deep PolyNet, designed following thi...",NaN,https://arxiv.org/abs/1611.05725,282.0,PolyNet: A Pursuit of Structural Diversity in ...,...,NaN,563.599706,178.564122,114.281038,68.692074,202.062472,31.682792,20.276987,12.188096,35.852125
1965,MoE-Multi,Language,"Language modeling,Translation","N Shazeer, A Mirhoseini, K Maziarz, A Davis","Highly cited,SOTA improvement","""On large language modeling and machine transl...",Unreleased,https://arxiv.org/abs/1701.06538,2037.0,Outrageously Large Neural Networks: The Sparse...,...,NaN,3538.189418,1519.646471,972.573741,584.594865,461.374341,42.949834,27.487894,16.522430,13.039843


In [34]:
filtered_component_cost_df.to_csv(results_dir + 'cost_components.csv', index=False)

In [35]:
# Average percentage for each component
filtered_component_cost_df[cost_component_pc_names].mean()

AI accelerator chip cost (%)           45.038389
Other server components cost (%)       29.130089
Cluster-level interconnect cost (%)    17.397544
Energy cost (%)                         8.433978
dtype: float64

In [36]:
fig = px.bar(
    filtered_component_cost_df,
    x='Model',
    y='Energy cost (%)',
    barmode='stack',
    # labels='Cost %',
    # text='Energy cost %',
)
# axis labels
fig.update_xaxes(title_text='Model')
fig.update_yaxes(title_text='Energy cost (% of amortized hardware CapEx + energy)')
# fig.update_layout(
#     legend=dict(
#         title_text='Cost component',
#         x=0.75,
#         y=0.05,
#     )
# )
# limits 0 to 100
fig.update_yaxes(range=[0, 30])
# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'energy_percentage')

fig.show()

In [37]:
fig = px.scatter(
    filtered_component_cost_df,
    x='Publication date',
    y='Energy cost',
    text='Model',
)
# axis labels
fig.update_xaxes(title_text='Model')
fig.update_yaxes(title_text='Energy cost')
# log y
fig.update_yaxes(type='log')
# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'energy_cost')

fig.show()

In [38]:
from energy import energy_price

# Stacked bar chart of cost components, using component_cost_df
filtered_component_cost_df.loc[:, 'Energy (kWh)'] = [
    row['Energy cost'] / energy_price(row['Publication date'].year) 
    for _, row in filtered_component_cost_df.iterrows()
]
fig = px.scatter(
    filtered_component_cost_df,
    x='Publication date',
    y='Energy (kWh)',
    text='Model',
)
# log y
fig.update_yaxes(type='log')
# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'energy_kwh')

fig.show()

In [39]:
filtered_component_cost_df.columns

Index(['Model', 'Domain', 'Task', 'Authors', 'Notability criteria',
       'Notability criteria notes', 'Model accessibility', 'Link', 'Citations',
       'Reference',
       ...
       'Cost', 'AI accelerator chip cost', 'Other server components cost',
       'Cluster-level interconnect cost', 'Energy cost',
       'AI accelerator chip cost (%)', 'Other server components cost (%)',
       'Cluster-level interconnect cost (%)', 'Energy cost (%)',
       'Energy (kWh)'],
      dtype='object', length=108)

In [40]:
filtered_component_cost_df = filtered_component_cost_df.dropna(subset=['Training hardware'])
power_col = 'Power capacity for final training run (kW)'
filtered_component_cost_df.loc[:, power_col] = [
    cluster_power_capacity(row['Training hardware'], row['Hardware quantity'], hardware_df, row['Organization'])
    for _, row in filtered_component_cost_df.iterrows()
]

fig = px.scatter(
    filtered_component_cost_df,
    x='Publication date',
    y=power_col,
    text='Model',
)
# log y
fig.update_yaxes(type='log')
# size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=40, b=10))

save_plot(fig, results_dir, 'power_capacity_kw')

fig.show()

In [41]:
filtered_component_cost_df['Publication date (float)'] = datetime_to_float_year(
    pd.to_datetime(filtered_component_cost_df['Publication date'])
)

In [42]:
power_reg_results = fit_ols_regression(
    filtered_component_cost_df,
    ['Publication date (float)'],
    power_col,
    logy=True
)
power_reg_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.863
Model:                            OLS   Adj. R-squared:                  0.859
Method:                 Least Squares   F-statistic:                     233.0
Date:                Thu, 13 Mar 2025   Prob (F-statistic):           1.51e-17
Time:                        17:02:39   Log-Likelihood:                -10.518
No. Observations:                  39   AIC:                             25.04
Df Residuals:                      37   BIC:                             28.36
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       -717.4255     47.192    -15.202      0.000    -813.045    -621.806
x1             0.3564      0.023     15.263      0.000       0.309       0.404
==============================================================================
Omnibus:                        5.566   Durbin-Watson:                   1.953
Prob(Omnibus):                  0.062   Jarque-Bera (JB):                4.658
Skew:                           0.514   Prob(JB):                       0.0974
Kurtosis:                       4.345   Cond. No.                     1.83e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.83e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [43]:
with open(f'{results_dir}/power_regression_results.out', 'w') as f:
    with redirect_stdout(f):
        print_growth_rates(power_reg_results)
print_growth_rates(power_reg_results)

N=39.0
R^2=0.86
0.3564433491873348 OOMs/year (90% CI: 0.3170440986156087, 0.39584259975906083)
2.2721832248227765x/year (90% CI: 2.075124216561464x, 2.4879554515155515x)
doubling time of 10.13445742837872 months (90% CI: 9.12574834079638, 11.393872220745795)


In [44]:
pred_start_year = 2015
pred_end_year = 2025
pred_start_date = f'{pred_start_year}-01-01'
pred_end_date = f'{pred_end_year}-01-01'

pred_years = pd.DataFrame({'Publication date (float)': np.linspace(pred_start_year, pred_end_year, 100)})
pred_years

,Publication date (float)
0,2015.00000
1,2015.10101
2,2015.20202
3,2015.30303
4,2015.40404
...,...
95,2024.59596
96,2024.69697
97,2024.79798
98,2024.89899


In [45]:
predicted_power_df = get_predictions(power_reg_results, pred_years, ['Publication date (float)'])
predicted_power_df['Publication date'] = predicted_power_df['Publication date (float)'].apply(float_year_to_datetime)
predicted_power_df

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper,Publication date (float),Publication date
0,0.807834,0.144451,0.564132,1.051537,0.207307,1.408361,2015.00000,2015-01-01
1,0.843838,0.142253,0.603844,1.083833,0.244807,1.442870,2015.10101,2015-02-06
2,0.879843,0.140061,0.643547,1.116139,0.282284,1.477402,2015.20202,2015-03-15
3,0.915847,0.137874,0.683241,1.148454,0.319737,1.511957,2015.30303,2015-04-21
4,0.951852,0.135693,0.722925,1.180778,0.357168,1.546535,2015.40404,2015-05-28
...,...,...,...,...,...,...,...,...
95,4.228250,0.103442,4.053734,4.402766,3.652318,4.804182,2024.59596,2024-08-06
96,4.264254,0.105487,4.086289,4.442220,3.687268,4.841241,2024.69697,2024-09-12
97,4.300259,0.107544,4.118822,4.481696,3.722193,4.878325,2024.79798,2024-10-19
98,4.336263,0.109614,4.151335,4.521192,3.757091,4.915435,2024.89899,2024-11-25


## Power plot

In [46]:
label_systems = ['GNMT', 'AlphaGo Master', 'AlphaGo Zero', 'AlphaZero', 'DALL-E', 'GPT-3 175B (davinci)', 'PaLM (540B)', 'Llama 2-70B', 'Falcon 180B', 'GPT-4', 'Gemini 1.0 Ultra', 'Inflection-2']

fig = px.scatter(
    filtered_component_cost_df,
    x='Publication date',
    y=power_col,
    log_y=True,
)

# Marker color
fig.update_traces(
    marker=dict(
        color='rgb(0,100,200)',
    ),
    selector=dict(mode='markers'),
)

fig.add_scatter(
    x=filtered_component_cost_df.loc[filtered_component_cost_df['Model'].isin(label_systems)]['Publication date'],
    y=filtered_component_cost_df.loc[filtered_component_cost_df['Model'].isin(label_systems)][power_col],
    text=filtered_component_cost_df.loc[filtered_component_cost_df['Model'].isin(label_systems)]['Model'],
    mode='text',
    showlegend=False,
)

# Shade in CI
fig.add_scatter(
    x=predicted_power_df['Publication date'],
    y=10**predicted_power_df['mean_ci_lower'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
)
fig.add_scatter(
    x=predicted_power_df['Publication date'],
    y=10**predicted_power_df['mean_ci_upper'],
    mode='lines',
    fill='tonexty',
    fillcolor='rgba(0,100,200,0.2)',
    line=dict(width=0),
    name='90% CI of mean',
)
fig.add_scatter(
    x=predicted_power_df['Publication date'],
    y=10**predicted_power_df['mean'],
    mode='lines',
    line=dict(color='rgb(0,100,200)'),
    name=f'Regression mean (growth rate: {10**power_reg_results.params[1]:.1f}x per year)',
)

fig.update_traces(textposition='top center')

# axis limits
fig.update_xaxes(range=[pred_start_date, pred_end_date])
# fig.update_xaxes(range=['2015-01-01', '2025-01-01'])  # manual
# fig.update_yaxes(range=[1, 6])

# legend on bottom-right of the axes
fig.update_layout(
    legend=dict(
        x=0.45,
        y=0.05,
    )
)

# axis labels
fig.update_xaxes(title_text='Publication date')
fig.update_yaxes(title_text='Power (kW, log scale)')

# title
fig.update_layout(title_text='Cluster power required for final training run')

# update size
fig.update_layout(
    autosize=False,
    width=800,
    height=600,
    title_font=dict(
        size=16,
    ),
    title_x=0.5,
)

# font size
fig.update_layout(
    font=dict(
        size=14,
    )
)

# margins
fig.update_layout(margin=dict(l=10, r=10, t=60, b=10))

save_plot(fig, results_dir, 'power_regression')

fig.show()